<a href="https://colab.research.google.com/github/RahulSingh85/AOA/blob/main/Text%20to%20Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


GPU Available: True
GPU Name: Tesla T4


In [3]:
!pip install torch torchvision torchaudio diffusers transformers accelerate opencv-python ffmpeg-python


In [4]:
import os
import torch
from diffusers import StableDiffusionPipeline
import cv2
import glob
import numpy as np
from moviepy.video.io.ImageSequenceClip import ImageSequenceClip
from moviepy.video.fx.all import fadein, fadeout
from moviepy.editor import AudioFileClip, CompositeAudioClip

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load Stable Diffusion Model
try:
    pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
    pipeline.to(device)
    print("Model loaded successfully!")
except Exception as e:
    print("Error loading model:", e)
    exit()

# Get user input
text_prompt = input("Enter text for AI video generation: ")
if not text_prompt.strip():
    print("Error: Text prompt cannot be empty!")
    exit()

# Create output directory
output_dir = "output_images"
os.makedirs(output_dir, exist_ok=True)

# Generate images
num_frames = 20  # Increased number of images for smoother video
fps = 10  # Higher FPS for smoother video
image_paths = []

for i in range(num_frames):
    print(f"Generating frame {i+1}/{num_frames}...")
    try:
        image = pipeline(text_prompt).images[0]
        image_path = os.path.join(output_dir, f"frame_{i:03d}.png")
        image.save(image_path)
        image_paths.append(image_path)
    except Exception as e:
        print(f"Error generating frame {i+1}:", e)
        continue

# Verify if images were generated
if not image_paths:
    print("Error: No images were generated!")
    exit()

print("Images generated successfully:", len(image_paths))

# Apply motion interpolation to smooth transitions
def interpolate_images(image_paths, output_dir):
    interpolated_paths = []
    for i in range(len(image_paths) - 1):
        img1 = cv2.imread(image_paths[i])
        img2 = cv2.imread(image_paths[i + 1])

        if img1 is None or img2 is None:
            print(f"Skipping interpolation for frame {i} due to missing images.")
            continue

        for alpha in np.linspace(0, 1, 5):  # Generate 5 intermediate frames
            blended = cv2.addWeighted(img1, 1 - alpha, img2, alpha, 0)
            interp_path = os.path.join(output_dir, f"interp_{i}_{int(alpha*10)}.png")
            cv2.imwrite(interp_path, blended)
            interpolated_paths.append(interp_path)
    return sorted(image_paths + interpolated_paths)

image_paths = interpolate_images(image_paths, output_dir)
print("Total frames after interpolation:", len(image_paths))

# Convert images to video with transitions
def create_video(image_paths, output_filename, fps, audio_path=None):
    if not image_paths:
        print("Error: No images available to create video!")
        return

    clip = ImageSequenceClip(image_paths, fps=fps)
    clip = fadein(clip, 0.5)
    clip = fadeout(clip, 0.5)

    # Add background music if provided
    if audio_path and os.path.exists(audio_path):
        try:
            audio = AudioFileClip(audio_path)
            audio = audio.set_duration(clip.duration)
            audio = audio.volumex(0.7)  # Reduce volume to 70%
            clip = clip.set_audio(audio)
        except Exception as e:
            print("Error loading audio file:", e)

    clip.write_videofile(output_filename, codec="libx264", audio_codec="aac", bitrate="2000k")

# Define audio file path (user can replace this with their own audio file)
audio_file = "background_music.mp3"  # Ensure this file exists in the working directory

video_filename = "output_video.mp4"
create_video(image_paths, video_filename, fps, audio_file)

print("Video generation complete! Saved as:", video_filename)


  if event.key is 'enter':



Using device: cuda


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Model loaded successfully!
Enter text for AI video generation: Anime fight
Generating frame 1/20...


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
!pip install diffusers transformers accelerate safetensors
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install numpy opencv-python moviepy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!mkdir -p /content/drive/MyDrive/stable-diffusion


In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_path = "/content/drive/MyDrive/stable-diffusion/v1-5-pruned.safetensors"  # Change to your local path
pipeline = StableDiffusionPipeline.from_pretrained(model_path)
pipeline.to("cuda" if torch.cuda.is_available() else "cpu")

print("Model loaded successfully!")
